In [1]:
from data_loader import load_data
import numpy as np
import math
import random

In [ ]:
def greedy_solution(n_depots, n_customers, capacities, setup_costs, demands, costs):
    customer_assignment = [-1] * n_customers
    remaining_capacity = capacities.copy()
    open_depots = set()

    for c in range(n_customers):
        min_cost = float('inf')
        best_depot = -1
        for d in range(n_depots):
            if remaining_capacity[d] >= demands[c] and costs[c][d] < min_cost:
                min_cost = costs[c][d]
                best_depot = d
        if best_depot == -1:
            raise ValueError("No feasible assignment found")
        customer_assignment[c] = best_depot
        remaining_capacity[best_depot] -= demands[c]
        open_depots.add(best_depot)

    total_cost = sum(setup_costs[d] for d in open_depots)
    total_cost += sum(costs[c][customer_assignment[c]] for c in range(n_customers))
    return list(open_depots), customer_assignment, total_cost

def calculate_cost(open_depots, assignment, setup_costs, costs):
    setup = sum(setup_costs[d] for d in open_depots)
    transport = sum(costs[i][assignment[i]] for i in range(len(assignment)))
    return setup + transport

def simulated_annealing(n_depots, n_customers, capacities, setup_costs, demands, costs,
                        initial_temp=1000, final_temp=1, alpha=0.95, max_iter=1000):
    open_depots, assignment, cost = greedy_solution(n_depots, n_customers, capacities, setup_costs, demands, costs)
    current_open = open_depots.copy()
    current_assignment = assignment.copy()
    current_cost = cost
    best_open = current_open.copy()
    best_assignment = current_assignment.copy()
    best_cost = current_cost

    temp = initial_temp
    while temp > final_temp:
        for _ in range(max_iter):
            new_open = current_open.copy()
            new_assignment = current_assignment.copy()

            # Rastgele bir müşteri seç ve başka bir geçerli depoya ata
            c = random.randint(0, n_customers - 1)
            current_depot = new_assignment[c]
            possible_depots = [d for d in range(n_depots) if d != current_depot and capacities[d] >= demands[c]]
            if not possible_depots:
                continue
            new_depot = random.choice(possible_depots)

            # kapasiteyi güncelle
            depot_capacities = capacities.copy()
            for i in range(n_customers):
                if i != c:
                    depot_capacities[new_assignment[i]] -= demands[i]

            if depot_capacities[new_depot] < demands[c]:
                continue

            new_assignment[c] = new_depot
            new_open = set(new_assignment)
            new_cost = calculate_cost(new_open, new_assignment, setup_costs, costs)

            delta = new_cost - current_cost
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_open = new_open
                current_assignment = new_assignment
                current_cost = new_cost
                if current_cost < best_cost:
                    best_open = current_open.copy()
                    best_assignment = current_assignment.copy()
                    best_cost = current_cost

        temp *= alpha

    return sorted(list(best_open)), best_assignment, best_cost


In [3]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [0, 1, 3, 5, 6, 7, 8, 10, 12, 16, 17, 19, 22, 23, 24]
Müşterilerin Depo Seçimi: [7, 6, 0, 24, 7, 0, 1, 5, 7, 7, 3, 10, 5, 0, 6, 7, 3, 8, 3, 6, 3, 6, 10, 0, 19, 10, 12, 10, 19, 0, 0, 10, 0, 16, 19, 19, 17, 5, 23, 24, 19, 3, 7, 6, 22, 23, 23, 6, 24, 19]
Toplam Maliyet: 796648.44


In [1]:
s = "7, 6, 0, 24, 7, 0, 1, 5, 7, 7, 3, 10, 5, 0, 6, 7, 3, 8, 3, 6, 3, 6, 10, 0, 19, 10, 12, 10, 19, 0, 0, 10, 0, 16, 19, 19, 17, 5, 23, 24, 19, 3, 7, 6, 22, 23, 23, 6, 24, 19"

s = s.replace(",", " ")
print(s)


7  6  0  24  7  0  1  5  7  7  3  10  5  0  6  7  3  8  3  6  3  6  10  0  19  10  12  10  19  0  0  10  0  16  19  19  17  5  23  24  19  3  7  6  22  23  23  6  24  19


In [4]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [2, 5, 10, 11, 17, 21, 22, 24, 26, 33, 36, 44, 45]
Müşterilerin Depo Seçimi: [24, 24, 2, 36, 24, 5, 36, 17, 24, 24, 10, 11, 22, 2, 24, 24, 10, 17, 10, 24, 10, 21, 22, 2, 24, 11, 26, 22, 22, 2, 5, 22, 5, 33, 24, 24, 36, 11, 45, 36, 22, 10, 24, 21, 44, 45, 45, 24, 36, 24]
Toplam Maliyet: 878329.9


In [2]:
s = "24, 24, 2, 36, 24, 5, 36, 17, 24, 24, 10, 11, 22, 2, 24, 24, 10, 17, 10, 24, 10, 21, 22, 2, 24, 11, 26, 22, 22, 2, 5, 22, 5, 33, 24, 24, 36, 11, 45, 36, 22, 10, 24, 21, 44, 45, 45, 24, 36, 24"

s = s.replace(",", " ")
print(s)


24  24  2  36  24  5  36  17  24  24  10  11  22  2  24  24  10  17  10  24  10  21  22  2  24  11  26  22  22  2  5  22  5  33  24  24  36  11  45  36  22  10  24  21  44  45  45  24  36  24


In [6]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [5, 12, 22, 39, 46, 49, 51, 52, 58, 67, 78, 88, 95, 115, 118, 129, 138, 144, 155, 156, 160, 163, 185, 199]
Müşterilerin Depo Seçimi: [118, 49, 78, 199, 129, 155, 78, 22, 155, 185, 163, 138, 51, 51, 39, 138, 163, 22, 115, 46, 156, 22, 95, 51, 67, 58, 160, 199, 129, 39, 67, 144, 67, 199, 199, 12, 49, 95, 160, 58, 12, 160, 52, 163, 88, 88, 12, 67, 88, 160, 78, 22, 52, 156, 138, 138, 144, 46, 51, 49, 163, 118, 22, 115, 51, 156, 49, 199, 88, 144, 12, 49, 12, 51, 78, 199, 46, 5, 199, 5, 185, 144, 185, 67, 46, 160, 5, 22, 138, 118, 52, 118, 67, 185, 95, 156, 129, 39, 49, 52, 22, 156, 88, 115, 129, 51, 129, 95, 46, 115, 78, 115, 155, 88, 46, 118, 199, 129, 12, 160, 118, 144, 12, 163, 46, 12, 144, 39, 95, 5, 95, 51, 12, 49, 51, 163, 155, 58, 88, 88, 22, 49, 46, 160, 95, 199, 12, 185, 155, 199, 163, 22, 160, 199, 78, 160, 144, 163, 39, 67, 12, 49, 67, 67, 51, 156, 12, 67, 46, 51, 160, 138, 185, 118, 39, 163, 138, 67, 129, 46, 22, 78, 46, 67, 22, 88, 51, 39, 88, 129, 78, 155, 144

In [3]:
s = "118, 49, 78, 199, 129, 155, 78, 22, 155, 185, 163, 138, 51, 51, 39, 138, 163, 22, 115, 46, 156, 22, 95, 51, 67, 58, 160, 199, 129, 39, 67, 144, 67, 199, 199, 12, 49, 95, 160, 58, 12, 160, 52, 163, 88, 88, 12, 67, 88, 160, 78, 22, 52, 156, 138, 138, 144, 46, 51, 49, 163, 118, 22, 115, 51, 156, 49, 199, 88, 144, 12, 49, 12, 51, 78, 199, 46, 5, 199, 5, 185, 144, 185, 67, 46, 160, 5, 22, 138, 118, 52, 118, 67, 185, 95, 156, 129, 39, 49, 52, 22, 156, 88, 115, 129, 51, 129, 95, 46, 115, 78, 115, 155, 88, 46, 118, 199, 129, 12, 160, 118, 144, 12, 163, 46, 12, 144, 39, 95, 5, 95, 51, 12, 49, 51, 163, 155, 58, 88, 88, 22, 49, 46, 160, 95, 199, 12, 185, 155, 199, 163, 22, 160, 199, 78, 160, 144, 163, 39, 67, 12, 49, 67, 67, 51, 156, 12, 67, 46, 51, 160, 138, 185, 118, 39, 163, 138, 67, 129, 46, 22, 78, 46, 67, 22, 88, 51, 39, 88, 129, 78, 155, 144, 118, 118, 78, 118, 51, 156, 118"

s = s.replace(",", " ")
print(s)


118  49  78  199  129  155  78  22  155  185  163  138  51  51  39  138  163  22  115  46  156  22  95  51  67  58  160  199  129  39  67  144  67  199  199  12  49  95  160  58  12  160  52  163  88  88  12  67  88  160  78  22  52  156  138  138  144  46  51  49  163  118  22  115  51  156  49  199  88  144  12  49  12  51  78  199  46  5  199  5  185  144  185  67  46  160  5  22  138  118  52  118  67  185  95  156  129  39  49  52  22  156  88  115  129  51  129  95  46  115  78  115  155  88  46  118  199  129  12  160  118  144  12  163  46  12  144  39  95  5  95  51  12  49  51  163  155  58  88  88  22  49  46  160  95  199  12  185  155  199  163  22  160  199  78  160  144  163  39  67  12  49  67  67  51  156  12  67  46  51  160  138  185  118  39  163  138  67  129  46  22  78  46  67  22  88  51  39  88  129  78  155  144  118  118  78  118  51  156  118


In [7]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [0, 5, 14, 26, 36, 38, 40, 44, 48, 52, 53, 58, 65, 69, 85, 105, 113, 120, 131, 162, 163, 176, 180, 182, 183, 185, 195, 204, 206, 220, 231, 248, 249, 252, 266, 269, 285, 287, 290, 293, 297]
Müşterilerin Depo Seçimi: [285, 182, 252, 36, 65, 195, 58, 65, 293, 249, 248, 290, 131, 293, 162, 113, 266, 131, 252, 266, 176, 40, 269, 85, 206, 293, 5, 0, 248, 248, 231, 185, 285, 26, 206, 65, 185, 204, 185, 53, 185, 249, 220, 195, 69, 53, 185, 52, 185, 44, 285, 69, 231, 293, 176, 249, 36, 248, 163, 206, 113, 38, 248, 131, 0, 69, 85, 0, 231, 0, 290, 183, 220, 204, 293, 69, 297, 185, 285, 0, 26, 65, 195, 14, 53, 269, 252, 249, 53, 297, 293, 287, 5, 231, 266, 0, 195, 52, 248, 285, 185, 38, 85, 204, 38, 38, 65, 290, 131, 162, 266, 231, 58, 204, 220, 52, 220, 120, 85, 252, 40, 163, 220, 182, 285, 26, 38, 38, 131, 69, 48, 38, 38, 52, 293, 65, 0, 131, 269, 44, 180, 65, 131, 5, 290, 48, 285, 65, 185, 69, 163, 293, 0, 176, 38, 36, 113, 290, 14, 204, 287, 183, 131, 185, 44, 176, 206, 5, 206

In [4]:
s = "285, 182, 252, 36, 65, 195, 58, 65, 293, 249, 248, 290, 131, 293, 162, 113, 266, 131, 252, 266, 176, 40, 269, 85, 206, 293, 5, 0, 248, 248, 231, 185, 285, 26, 206, 65, 185, 204, 185, 53, 185, 249, 220, 195, 69, 53, 185, 52, 185, 44, 285, 69, 231, 293, 176, 249, 36, 248, 163, 206, 113, 38, 248, 131, 0, 69, 85, 0, 231, 0, 290, 183, 220, 204, 293, 69, 297, 185, 285, 0, 26, 65, 195, 14, 53, 269, 252, 249, 53, 297, 293, 287, 5, 231, 266, 0, 195, 52, 248, 285, 185, 38, 85, 204, 38, 38, 65, 290, 131, 162, 266, 231, 58, 204, 220, 52, 220, 120, 85, 252, 40, 163, 220, 182, 285, 26, 38, 38, 131, 69, 48, 38, 38, 52, 293, 65, 0, 131, 269, 44, 180, 65, 131, 5, 290, 48, 285, 65, 185, 69, 163, 293, 0, 176, 38, 36, 113, 290, 14, 204, 287, 183, 131, 185, 44, 176, 206, 5, 206, 131, 113, 183, 287, 182, 58, 290, 44, 220, 65, 40, 36, 44, 206, 120, 38, 204, 105, 163, 5, 113, 204, 182, 204, 36, 248, 290, 249, 113, 185, 183, 36, 206, 120, 293, 185, 206, 285, 53, 285, 26, 182, 48, 204, 185, 206, 14, 38, 293, 14, 0, 297, 40, 180, 120, 249, 53, 248, 269, 266, 40, 120, 163, 5, 266, 105, 195, 176, 269, 162, 266, 48, 182, 176, 65, 290, 204, 182, 40, 266, 176, 120, 182, 5, 48, 176, 290, 163, 176, 220, 26, 85, 38, 36, 252, 40, 85, 293, 14, 180, 131, 69, 182, 58, 269, 231, 65, 44, 65, 290, 40, 180, 69, 231, 269, 85, 249, 36, 58, 204, 290, 180, 120, 58, 287, 0, 113, 231, 206, 183, 185"

s = s.replace(",", " ")
print(s)


285  182  252  36  65  195  58  65  293  249  248  290  131  293  162  113  266  131  252  266  176  40  269  85  206  293  5  0  248  248  231  185  285  26  206  65  185  204  185  53  185  249  220  195  69  53  185  52  185  44  285  69  231  293  176  249  36  248  163  206  113  38  248  131  0  69  85  0  231  0  290  183  220  204  293  69  297  185  285  0  26  65  195  14  53  269  252  249  53  297  293  287  5  231  266  0  195  52  248  285  185  38  85  204  38  38  65  290  131  162  266  231  58  204  220  52  220  120  85  252  40  163  220  182  285  26  38  38  131  69  48  38  38  52  293  65  0  131  269  44  180  65  131  5  290  48  285  65  185  69  163  293  0  176  38  36  113  290  14  204  287  183  131  185  44  176  206  5  206  131  113  183  287  182  58  290  44  220  65  40  36  44  206  120  38  204  105  163  5  113  204  182  204  36  248  290  249  113  185  183  36  206  120  293  185  206  285  53  285  26  182  48  204  185  206  14  38  293  14

In [8]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [22, 37, 39, 46, 55, 58, 61, 74, 87, 100, 106, 109, 110, 115, 116, 129, 130, 133, 146, 147, 150, 151, 162, 163, 168, 174, 181, 187, 198, 207, 210, 228, 229, 241, 247, 249, 255, 259, 262, 265, 266, 268, 270, 274, 278, 283, 294, 298, 299, 301, 304, 309, 313, 320, 329, 339, 345, 352, 358, 371, 374, 383, 395, 398, 399, 406, 417, 425, 438, 442, 444, 446, 450, 454, 458, 467, 489, 497]
Müşterilerin Depo Seçimi: [265, 425, 247, 39, 301, 181, 74, 489, 110, 181, 100, 425, 259, 450, 109, 259, 163, 298, 133, 345, 417, 207, 283, 383, 229, 299, 116, 438, 198, 438, 345, 115, 383, 259, 406, 458, 87, 109, 229, 497, 298, 210, 383, 61, 249, 298, 274, 265, 116, 39, 294, 444, 168, 247, 129, 39, 39, 55, 262, 304, 150, 278, 150, 228, 298, 383, 228, 241, 146, 37, 301, 37, 304, 151, 151, 116, 198, 309, 147, 37, 207, 329, 345, 497, 228, 150, 150, 489, 301, 270, 274, 294, 129, 442, 37, 444, 259, 395, 61, 458, 130, 442, 294, 278, 100, 345, 446, 274, 116, 162, 37, 299, 454, 58, 309, 130, 329, 229,

In [5]:
s = "265, 425, 247, 39, 301, 181, 74, 489, 110, 181, 100, 425, 259, 450, 109, 259, 163, 298, 133, 345, 417, 207, 283, 383, 229, 299, 116, 438, 198, 438, 345, 115, 383, 259, 406, 458, 87, 109, 229, 497, 298, 210, 383, 61, 249, 298, 274, 265, 116, 39, 294, 444, 168, 247, 129, 39, 39, 55, 262, 304, 150, 278, 150, 228, 298, 383, 228, 241, 146, 37, 301, 37, 304, 151, 151, 116, 198, 309, 147, 37, 207, 329, 345, 497, 228, 150, 150, 489, 301, 270, 274, 294, 129, 442, 37, 444, 259, 395, 61, 458, 130, 442, 294, 278, 100, 345, 446, 274, 116, 162, 37, 299, 454, 58, 309, 130, 329, 229, 249, 467, 255, 100, 187, 241, 115, 304, 458, 345, 309, 229, 229, 467, 115, 163, 329, 265, 115, 395, 116, 299, 450, 399, 320, 115, 438, 74, 395, 174, 274, 309, 265, 309, 133, 358, 37, 313, 299, 294, 249, 283, 151, 444, 417, 294, 58, 74, 151, 438, 207, 398, 383, 352, 259, 247, 454, 133, 446, 162, 417, 383, 249, 467, 247, 39, 329, 454, 444, 22, 417, 262, 207, 304, 450, 371, 181, 398, 444, 247, 106, 39, 268, 294, 147, 352, 274, 262, 358, 298, 489, 358, 383, 259, 374, 163, 249, 450, 313, 255, 298, 298, 22, 187, 298, 399, 274, 207, 309, 163, 46, 299, 151, 266, 438, 187, 345, 320, 58, 274, 489, 198, 150, 259, 39, 309, 110, 301, 198, 147, 46, 115, 313, 374, 467, 61, 294, 442, 489, 450, 61, 229, 133, 425, 438, 147, 371, 358, 74, 109, 39, 301, 174, 150, 283, 207, 228, 278, 210, 345, 110, 174, 339, 406, 162, 301, 358, 383, 458, 374, 174, 265, 313, 168, 371, 454, 210, 207, 301, 228, 55, 444, 241, 109, 383, 110, 371, 129, 450, 489, 266, 304, 278, 187, 39, 301, 383, 489, 150, 438, 425, 109, 320, 374, 22, 247, 100, 228, 309, 497, 198, 61, 249, 110, 301, 270, 37, 37, 352, 210, 417, 301, 151, 55, 110, 37, 329, 304, 228, 358, 450, 55, 133, 106, 87, 147, 130, 87, 283, 268, 130, 345, 299, 100, 163, 425, 229, 247, 162, 450, 446, 489, 406, 207, 371, 58, 116, 294, 100, 247, 37, 100, 39, 329, 151, 100, 162, 489, 129, 241, 299, 228, 37, 87, 313, 181, 151, 74, 489, 458, 270, 458, 39, 371, 395, 395, 58, 329, 37, 489, 259, 110, 198, 109, 174, 150, 58, 313, 61, 229, 444, 241, 442, 352, 301, 270, 22, 313, 489, 162, 106, 187, 425, 168, 371, 39, 262, 309, 442, 133, 249, 162, 265, 247, 259, 55, 383, 444, 425, 187, 406, 174, 467, 58, 417, 399, 352, 247, 274, 294, 55, 266, 489, 115, 298, 116, 151, 438, 37, 210, 241, 58, 339, 37, 438, 398, 100, 198, 299, 266, 109, 147, 274, 187, 255, 358, 329, 58, 37, 74, 442, 399, 399, 358, 129, 438, 174, 187, 116, 241, 198, 129"

s = s.replace(",", " ")
print(s)


265  425  247  39  301  181  74  489  110  181  100  425  259  450  109  259  163  298  133  345  417  207  283  383  229  299  116  438  198  438  345  115  383  259  406  458  87  109  229  497  298  210  383  61  249  298  274  265  116  39  294  444  168  247  129  39  39  55  262  304  150  278  150  228  298  383  228  241  146  37  301  37  304  151  151  116  198  309  147  37  207  329  345  497  228  150  150  489  301  270  274  294  129  442  37  444  259  395  61  458  130  442  294  278  100  345  446  274  116  162  37  299  454  58  309  130  329  229  249  467  255  100  187  241  115  304  458  345  309  229  229  467  115  163  329  265  115  395  116  299  450  399  320  115  438  74  395  174  274  309  265  309  133  358  37  313  299  294  249  283  151  444  417  294  58  74  151  438  207  398  383  352  259  247  454  133  446  162  417  383  249  467  247  39  329  454  444  22  417  262  207  304  450  371  181  398  444  247  106  39  268  294  147  352  27